# ADR Architecture Quick analysis

## Necessary code imports

In [1]:
!pip install bokeh
import numpy as np
import matplotlib.pyplot as plt
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
from bokeh.layouts import layout

## Architecture analysis

In [0]:
# earth assumptions
mu_Earth = 3.986004418e14 # [m³/s²] gravity constant
R_Earth = 6371000.0 # [m] radius

# SC assumptions
SC_a = 7074000.0 # [m] semi-major axis
SC_w = np.sqrt(mu_Earth/SC_a**3) # [1/s] angular frequency
SC_T = 2*np.pi/SC_w # [s] period

# cost assumptions
RC_per_chaser = 3e6 #$
RC_per_launch = 10e6 #$

NRC_development = 50e6 #$

revenue_per_target = 10e6 #$
launchs_per_year = 20 #-

development_time = 5 #year


In [0]:
# models

def hohmann(r1,r2,mu):
    v1 = np.sqrt(mu/r1)*(np.sqrt(2*r2/(r1+r2))-1)
    v2 = np.sqrt(mu/r2)*(1-np.sqrt(2*r1/(r1+r2)))
    return v1,v2
  
def NRC(year):
  NRC = NRC_development / development_time * year
  return NRC

def RC(year):
  if year > development_time:
    launch_cost = RC_per_launch * launchs_per_year * market_share * (year-development_time)
    chaser_cost = RC_per_chaser * chasers_per_launch * launchs_per_year * market_share * (year-development_time)
    RC = launch_cost + chaser_cost
    return RC
  else:
    return 0

def revenue(year):
  if year > development_time :
    return revenue_per_target * chasers_per_launch * launchs_per_year * market_share * (year-development_time)
  else:
    return 0

def benefit(year):
  return revenue(year)-RC(year)-NRC(year)

def debris(year):
  if year > development_time :
    if launch_generates_debris:
      return 23500 - (targets_per_chaser * chasers_per_launch - 1) * launchs_per_year * market_share * (year-development_time)
    else:
      return 23500 - targets_per_chaser * chasers_per_launch * launchs_per_year * market_share * (year-development_time)
  else:
    return 23500

In [3]:
#@title Architecture parameters { run: "auto", vertical-output: true }
#@markdown (Double click here to show code) { display-mode: "form" }

launch = "dedicated" #@param ["dedicated", "primary", "secondary"]
launch_altitude = 500 #@param {type:"integer"}
launch_a = R_Earth + launch_altitude * 1000

chaser_phasing_propulsion = "bi-propellant" #@param ["bi-propellant", "mono-propellant", "electrical"]
chaser_homing_propulsion = "bi-propellant" #@param ["bi-propellant", "mono-propellant", "electrical"]
chaser_rdv_propulsion = "bi-propellant" #@param ["bi-propellant", "mono-propellant", "electrical"]
chaser_reentry_propulsion = "bi-propellant" #@param ["bi-propellant", "mono-propellant", "electrical"]



# computation
time = np.linspace(1,10,10)

benefit = [benefit(t) for t in time]
debris = [debris(t) for t in time]

#plotting
output_notebook()

p1 = figure(toolbar_location="above", plot_width=400, plot_height=400)
p1.line(time, [b*1e-6 for b in benefit], color = 'green')
p1.xaxis.axis_label = 'Time [Year]'
p1.yaxis.axis_label = 'Cashflow [M$]'

p2 = figure(toolbar_location="above", plot_width=400, plot_height=400)
p2.line(time, debris, color = 'blue')
p2.xaxis.axis_label = 'Time [Year]'
p2.yaxis.axis_label = 'Debris [-]'

l = layout([
  [p1, p2],
])
show(l)

Loading BokehJS ...